In [ ]:
!pip install pandas keras tensorflow nltk scikit-learn

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
import nltk
import numpy as np


2024-09-19 23:40:54.831699: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 23:40:54.835164: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 23:40:54.844279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 23:40:54.858253: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 23:40:54.862617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 23:40:54.874695: I tensorflow/core/platform/cpu_feature_gu

In [2]:
import sys
sys.path.append('/workspaces/NLP-sesion')

In [3]:
from process_data import ProcessData

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [4]:
df=pd.read_csv("/workspaces/NLP-sesion/IMDBDataset.csv", nrows=20000)

df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})
processor = ProcessData(df, "review", "sentiment")

df = processor.data_cleaning()
X_train, X_test, y_train, y_test  = processor.data_split()
print(len(X_train), len(X_test), len(y_train), len(y_test))

/tmp/ipykernel_114003/2831058031.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})


16000 4000 16000 4000


In [5]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X_train['cleaned_review'])
vocab_size=len(tokenizer.word_index)+1
print(f'Vocab Size: {vocab_size}')

Vocab Size: 60714


In [6]:
max_words = 50

X_train=pad_sequences(tokenizer.texts_to_sequences(X_train['cleaned_review']), maxlen=max_words, padding="post")
X_test=pad_sequences(tokenizer.texts_to_sequences(X_test['cleaned_review']), maxlen=max_words, padding="post")


In [7]:
X_train[0]

array([  529,  4268,    17,    69,   945,  1203,    44,     5,    69,
        3993,    35,    81,    67,    97,   204,     6,    83,    50,
         630,    69,   324,   278,    21,    53,     1,  1477,     2,
          96,    31,     4,  3582,  6453,  7477,    37,   707,  1489,
          42,     4,  1397,    50,    27,     1,  2694,   950,   232,
          41,   659,    42, 11244,  6203], dtype=int32)

In [8]:
# Create an LSTM model with an Embedding layer and fit training data
model=Sequential()
model.add(layers.Embedding(input_dim=vocab_size,output_dim=100))
model.add(layers.Bidirectional(layers.LSTM(128)))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='BinaryCrossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=3, epochs=2,validation_data=(X_test,y_test), verbose=True)

Epoch 1/2


2024-09-19 23:44:00.438829: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24285600 exceeds 10% of free system memory.
2024-09-19 23:44:00.503580: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24285600 exceeds 10% of free system memory.
2024-09-19 23:44:00.517231: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24285600 exceeds 10% of free system memory.
2024-09-19 23:44:00.569245: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24285600 exceeds 10% of free system memory.
2024-09-19 23:44:00.588738: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24285600 exceeds 10% of free system memory.


5334/5334 ━━━━━━━━━━━━━━━━━━━━ 382s 71ms/step - accuracy: 0.7002 - loss: 0.5575 - val_accuracy: 0.8108 - val_loss: 0.4072
Epoch 2/2
5334/5334 ━━━━━━━━━━━━━━━━━━━━ 392s 73ms/step - accuracy: 0.8906 - loss: 0.2627 - val_accuracy: 0.8217 - val_loss: 0.4260


In [50]:
pd.set_option('display.max_colwidth', None)
sentence_to_test = "It was not painful to watch, actually i thought it was an ok movie"
sentiment = 1

df = pd.DataFrame([[sentence_to_test, sentiment]], columns=['review', 'sentiment'])
test_processor = ProcessData(df, "review", "sentiment")

cleaned_sentence = test_processor.data_cleaning()
sentence_seq = tokenizer.texts_to_sequences(cleaned_sentence["cleaned_review"])  # convert to sequence
preprocessed_sentence = pad_sequences(sentence_seq, maxlen=50, padding="post")
len(preprocessed_sentence)
print(cleaned_sentence['cleaned_review'])

0    it be not painful to watch , actually i think it be an ok movie
Name: cleaned_review, dtype: object


In [51]:
sentence_seq[0]

[9, 2, 21, 1301, 6, 58, 161, 10, 71, 9, 2, 32, 594, 14]

In [52]:
# Make a prediction
prediction = model(preprocessed_sentence, training=False)

prediction.numpy()[0]

array([0.05127384], dtype=float32)